In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：用于在线预测的本地表格回归模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_local_tabular_regression_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_local_tabular_regression_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概览

本教程演示了如何使用Python的Vertex客户端库，部署一个在本地训练的自定义表格回归模型，以进行在线预测。

数据集

本教程中使用的数据集是[波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。在本教程中使用的数据集版本已内置于TensorFlow中。训练好的模型用来预测房屋价格的中位数，单位为1K美元。

### 目标

在这个笔记本中，您将在本地创建一个自定义模型，然后学习如何将本地训练的模型部署到Vertex，然后对部署的模型进行预测。您也可以使用`gcloud`命令行工具或在线使用Google Cloud Console来创建和部署模型。

所执行的步骤包括：

- 在本地创建一个模型。
- 在本地训练模型。
- 查看模型评估。
- 将模型上传为Vertex `Model`资源。
- 将`Model`资源部署到服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`资源。

### 费用

本教程使用了谷歌云(GCP)的计费组件：

* Vertex AI
* 云存储

了解[Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和[云存储
定价](https://cloud.google.com/storage/pricing)，并使用[定价
计算器](https://cloud.google.com/products/calculator/)
根据您预计的使用情况生成费用估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动核心

一旦您安装了Vertex客户端库和Google *cloud-storage*，您需要重新启动笔记本核心，以便它可以找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果有此选项，请确保在 GPU 运行时中运行本笔记本。在 Colab 中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的 Google Cloud 项目

**无论您的笔记本环境如何，都需要执行以下步骤。**

1. [选择或创建 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。首次创建帐户时，您将获得 $300 的免费信用用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex APIs 和 Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Google Cloud Notebook 中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**: Jupyter 将以 `!` 为前缀的行作为 shell 命令运行，并将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改`REGION`变量，该变量用于本笔记本其余部分的操作。以下是Vertex支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法在Vertex中使用多区域存储桶进行训练。并非所有区域都支持所有Vertex服务。有关各区域的最新支持信息，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您在直播教程会话中，您可能会使用共享的测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您需要为每个实例会话创建一个时间戳，并将其附加到在本教程中将要创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###验证您的Google Cloud帐户

**如果您正在使用Google Cloud Notebook**，您的环境已经经过验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并在提示时按照说明进行帐户认证。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入名称，然后点击**创建**。

在**授予此服务帐户对项目的访问权限**部分，点击角色下拉列表。在筛选框中输入“Vertex”，然后选择**Vertex管理员**。在筛选框中输入“存储对象管理员”，然后选择**Storage Object Admin**。

点击创建。一个包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中将您的服务帐户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您的笔记本环境如何，都需要执行以下步骤。**

当您使用 Vertex 客户端库提交一个自定义训练作业时，您需要将包含训练代码的 Python 软件包上传到一个云存储桶中。Vertex会从该软件包中运行代码。在本教程中，Vertex还会将作业产生的训练模型保存在同一个存储桶中。然后，您可以基于这个输出创建一个 `Endpoint` 资源，以便提供在线预测。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有Google云项目中是全局唯一的，包括组织外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时才运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

#### 导入 Vertex 客户端库

将 Vertex 客户端库导入到我们的 Python 环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为Vertex设置以下常量：

- `API_ENDPOINT`：Vertex API服务的数据集、模型、作业、管道和端点服务的终端点。
- `PARENT`：数据集、模型、作业、管道和端点资源的Vertex位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

硬件加速器

为预测设置硬件加速器（例如 GPU）。

设置变量 `DEPLOY_GPU/DEPLOY_NGPU` 以使用支持 GPU 的容器映像，并为虚拟机（VM）实例分配的 GPU 数量。例如，要使用一个 GPU 容器映像，为每个 VM 分配 4 个 Nvidia Telsa K80 GPU，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于 GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，请指定 `(None, None)` 以使用一个容器映像在 CPU 上运行。

In [ ]:
if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

容器（Docker）映像

接下来，我们将设置用于预测的Docker容器映像

- 将变量`TF`设置为容器映像的TensorFlow版本。例如，`2-1`将是版本2.1，`1-15`将是版本1.15。以下列表显示了一些可用的预建映像：

 - TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
 - TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
 - TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
 - TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
 - XGBoost
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
 - Scikit-learn
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

有关最新列表，请查看[用于预测的预建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU)

#### 机器类型

接下来，设置用于预测的机器类型。

- 设置变量`DEPLOY_COMPUTE`以配置用于预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存。
     - `n1-highcpu`：每个vCPU 0.9GB内存。
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在，您已经准备好开始在本地对波士顿房屋进行自定义模型培训，然后部署该模型到云端。

## 设置客户端

Vertex 客户端库采用客户机/服务器模型。在您的一侧（Python 脚本）上，您将创建一个客户端，用于向 Vertex 服务器发送请求并接收响应。

在本教程中，您将针对工作流程中的不同步骤使用不同的客户端。因此，请预先设置所有客户端。

- 用于 `Model` 资源的模型服务。
- 用于部署的端点服务。
- 用于提供服务的预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

在本教程中，您可以在本地训练一个波士顿房价模型。

### 设置用于存储经过训练的模型的位置

您设置`MODEL_DIR`变量，指定在您的云存储桶中保存TensorFlow SavedModel格式的模型的位置。

此外，您还为训练脚本创建了本地文件夹。

In [ ]:
MODEL_DIR = BUCKET_NAME + "/boston"
model_path_to_deploy = MODEL_DIR

! rm -rf custom
! mkdir custom
! mkdir custom/trainer

#### Task.py 的内容

在下一个单元格中，您可以编写训练脚本 task.py 的内容。我不会详细阐述，只是为了让您查看。总结如下：

- 从命令行获取要保存模型工件的目录（`--model_dir`），如果没有指定，则从环境变量 `AIP_MODEL_DIR` 获取。
- 从 TF.Keras 内置数据集加载波士顿房屋数据集。
- 使用 TF.Keras 模型 API 构建简单的深度神经网络模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分布策略。
- 训练模型（`fit()`）并指定训练轮数为 `args.epochs`。
- 将训练好的模型保存到指定的模型目录（`save(args.model_dir)`）。
- 将每个特征的最大值保存到指定的参数文件中 `f.write(str(params))`。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_dnn_model()

# Train the model
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)
model.save(args.model_dir)

### 训练模型

In [ ]:
! python custom/trainer/task.py --epochs=10 --model-dir=$MODEL_DIR

加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 存储桶中。现在从云存储桶中加载它，然后您可以做一些事情，比如评估这个模型，进行预测。

要加载模型，您可以使用 TF.Keras 中的 `model.load_model()` 方法，将云存储路径传递给它，该路径指定为 `MODEL_DIR`。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在让我们找出模型的表现如何。

### 载入评估数据

您将从`tf.keras.datasets`中载入波士顿房屋测试（留出）数据，使用`load_data()`方法。这将返回一个包含两个元素的元组数据集。第一个元素是训练数据，第二个是测试数据。每个元素也是一个包含两个元素的元组：特征数据和对应的标签（自住房屋的中位价）。

您不需要用到训练数据，因此我们将其载入为`(_,_)`。

在运行数据进行评估之前，您需要对其进行预处理：

x_test:
1. 对每列数据进行标准化（重缩放），即将每个数值除以该列的最大值。这将用32位浮点数值在0和1之间替代每个单独的数值。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import boston_housing

(_, _), (x_test, y_test) = boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)


def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float32)
    return feature


# Let's save one data item that has not been scaled
x_test_notscaled = x_test[0:1].copy()

for _ in range(13):
    x_test[_] = scale(x_test[_])
x_test = x_test.astype(np.float32)

print(x_test.shape, x_test.dtype, y_test.shape)
print("scaled", x_test[0])
print("unscaled", x_test_notscaled)

### 进行模型评估

现在评估一下定制作业中的模型表现如何。

In [ ]:
model.evaluate(x_test, y_test)

## 上传模型进行服务

接下来，您将把 TF.Keras 模型从自定义作业上传到 Vertex `Model` 服务，这将为您的自定义模型创建一个 Vertex `Model` 资源。在上传过程中，您需要定义一个服务函数，将数据转换为模型期望的格式。如果您将编码数据发送到 Vertex，您的服务函数将确保在将数据传递到模型输入之前，在模型服务器上对数据进行解码。

### 服务函数如何工作

当您向在线预测服务器发送请求时，该请求将被 HTTP 服务器接收。HTTP 服务器从 HTTP 请求内容体中提取预测请求。提取的预测请求将被转发到服务函数。对于 Google 预构建的预测容器，请求内容被作为 `tf.string` 传递给服务函数。

服务函数由两部分组成：

- `预处理函数`:
  - 将输入 (`tf.string`) 转换为底层模型 (动态图) 的输入形状和数据类型。
  - 执行与训练底层模型期间相同的数据预处理 -- 例如，归一化、缩放等。
- `后处理函数`:
  - 将模型输出转换为接收应用程序期望的格式 -- 例如，压缩输出。
  - 为接收应用程序打包输出 -- 例如，添加标题，制作 JSON 对象等。

预处理和后处理函数均被转换为静态图，然后与模型融合。底层模型的输出传递给后处理函数。后处理函数将转换/打包后的输出传递回 HTTP 服务器。HTTP 服务器将输出作为 HTTP 响应内容返回。

在构建 TF.Keras 模型的服务函数时，您需要考虑一个问题，即它们作为静态图运行。这意味着，您不能使用需要动态图的 TF 图运算。如果您这样做，您将在服务函数编译期间收到一个错误，指示您正在使用不受支持的 EagerTensor。

获取服务函数签名

您可以通过重新加载模型到内存，查询每个层对应的签名来获取模型输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务函数而不是模型，因此您需要知道服务函数的输入层名称 -- 这将在以后进行预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

### 上传模型

使用这个辅助函数`upload_model`来上传你的模型，保存在SavedModel格式中，到`Model`服务中，该服务将为你的模型实例化一个Vertex `Model`资源实例。一旦你完成了这一步，你可以像对待其他Vertex `Model`资源实例一样使用`Model`资源实例，比如部署到用于提供预测的`Endpoint`资源中。

这个辅助函数接受以下参数：

- `display_name`：用于`Endpoint`服务的可读名称。
- `image_uri`：模型部署的容器镜像。
- `model_uri`：我们SavedModel艺术品在Cloud Storage中的路径。在这个教程中，这是`trainer/task.py`保存模型艺术品的Cloud Storage位置，我们在变量`MODEL_DIR`中指定。

这个辅助函数调用`Model`客户端服务的方法`upload_model`，该方法接受以下参数：

- `parent`：`Dataset`、`Model`和`Endpoint`资源的Vertex根路径。
- `model`：Vertex `Model`资源实例的规范。

现在让我们深入了解Vertex模型规范`model`。这是一个字典对象，包含以下字段：

- `display_name`：`Model`资源的可读名称。
- `metadata_schema_uri`：由于您的模型没有使用Vertex `Dataset`资源构建，您将留空（`''`）。
- `artificat_uri`：模型以SavedModel格式存储在Cloud Storage路径。
- `container_spec`：这是安装在`Endpoint`资源上的Docker容器的规范，从这个容器中，`Model`资源将提供预测。使用之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。

将模型上传到Vertex Model资源将返回一个长时间运行的操作，因为可能需要一些时间。你调用response.result()，这是一个同步调用，在Vertex Model资源准备就绪时将返回。

辅助函数返回相应Vertex Model实例的完全限定标识符`upload_model_response.model`。你将保存这个标识符，以便在后续步骤中使用。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):
    model = {
        "display_name": display_name,
        "metadata_schema_uri": "",
        "artifact_uri": model_uri,
        "container_spec": {
            "image_uri": image_uri,
            "command": [],
            "args": [],
            "env": [{"name": "env_name", "value": "env_value"}],
            "ports": [{"container_port": 8080}],
            "predict_route": "",
            "health_route": "",
        },
    }
    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model(
    "boston-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy
)

获取“Model”资源信息

现在让我们仅获取您的模型的模型信息。使用这个辅助函数`get_model`，带有以下参数：

- `name`：用于“Model”资源的Vertex唯一标识符。

这个辅助函数调用Vertex的`Model`客户端服务方法`get_model`，带有以下参数：

- `name`：用于“Model”资源的Vertex唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

部署`Model`资源

现在部署经过训练的Vertex自定义`Model`资源。这需要两个步骤：

1. 创建一个用于部署`Model`资源的`Endpoint`资源。

2. 将`Model`资源部署到`Endpoint`资源上。

### 创建一个`Endpoint`资源

使用这个辅助函数`create_endpoint`来创建一个用于部署模型以提供预测的终端节点，需要传入以下参数：

- `display_name`：一个可读性强的`Endpoint`资源的名称。

该辅助函数使用终端节点客户端服务的`create_endpoint`方法，该方法需要传入以下参数:

- `display_name`：一个用于`Endpoint`资源的可读性强的名称。

创建一个`Endpoint`资源会返回一个长时间运行的操作，因为可能需要一段时间来为服务预测的`Endpoint`资源进行配置。当终端节点资源准备好后，您可以调用`response.result()`进行同步调用，该方法会在`Endpoint`资源就绪时返回。该辅助函数返回`Endpoint`资源的 Vertex 完全限定标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "boston_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的`端点`资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例扩展

处理在线预测请求时，您有几种选择可以对计算实例进行扩展：

- 单个实例：在线预测请求在单个计算实例上进行处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展：在线预测请求在您手动指定的固定数量的计算实例上进行拆分。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为与节点数量相同。当模型首次部署到实例时，固定数量的计算实例会被部署，并且在线预测请求会均匀分布在它们之间。

- 自动扩展：在线预测请求在可伸缩数量的计算实例上进行拆分。
  - 设置计算实例的最小（`MIN_NODES`）数量，在模型首次部署时进行部署和取消部署，并将最大（`MAX_NODES`）数量设置为根据负载情况进行部署。

计算实例的最小数量对应于字段`min_replica_count`，计算实例的最大数量对应于字段`max_replica_count`，在您提交之后的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署 `Model` 资源到 `Endpoint` 资源

使用这个辅助函数 `deploy_model` 来将从训练管道中上传（部署）的 `Model` 资源部署到为提供预测而创建的 `Endpoint` 资源，使用以下参数：

- `model`: 要上传（部署）的模型的 Vertex 完全合格的模型标识符。
- `deploy_model_display_name`: 部署模型的用户可读名称。
- `endpoint`: 要将模型部署到的 Vertex 完全合格的终端点标识符。

辅助函数调用 `Endpoint` 客户端服务的方法 `deploy_model`，该方法接受以下参数：

- `endpoint`: 要将 `Model` 资源部署到的 Vertex 完全合格的 `Endpoint` 资源标识符。
- `deployed_model`: 部署模型的要求规范。
- `traffic_split`: 流量分配到端点的百分比，指定为一个或多个键/值对的字典。
   - 如果只有一个模型，则指定为 **{ "0": 100 }**，其中 "0" 指此模型被上传并且 100 表示流量的 100%。
   - 如果端点上存在正在分配流量的现有模型，则使用 `model_id` 指定为 **{ "0": percent, model_id: percent, ... }**，其中 `model_id` 是现有模型的模型 ID，用于部署到端点。百分比必须总和为 100。

现在让我们深入了解 `deployed_model` 参数。此参数指定为 Python 字典，具有最少所需字段：

- `model`: 要部署的（上传）模型的 Vertex 完全合格模型标识符。
- `display_name`: 部署模型的用户可读名称。
- `disable_container_logging`: 这会禁用容器事件的日志记录，例如执行失败（默认情况下，容器日志记录已启用）。在调试部署时通常启用容器日志记录，然后在生产环境部署时禁用。
- `dedicated_resources`: 这指的是为提供预测请求而缩放的计算实例（副本）的数量。
  - `machine_spec`: 要提供的计算实例。使用之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只分配 CPU。
  - `min_replica_count`: 最初要提供的计算实例数，您之前将其设置为变量 `MIN_NODES`。
  - `max_replica_count`: 要扩展到的计算实例的最大数量，您之前将其设置为变量 `MAX_NODES`。

#### 流量分配

现在让我们深入了解 `traffic_split` 参数。此参数指定为 Python 字典。这可能起初会有点令人困惑。让我解释一下，您可以在一个端点上部署多个模型实例，然后设置每个实例接收的流量比例（百分比）。

为什么这样做？也许您已经在生产中部署了先前版本 -- 让我们称之为 v1。您在 v2 上获得了更好的模型评估，但在部署到生产环境之前，您并不能确定它是否真的更好。因此，在流量分配的情况下，您可能希望将 v2 部署到与 v1 相同的端点，但只有 10% 的流量分配给它。这样，您就可以在不干扰大多数用户的情况下监控其性能 -- 直到做出最终决定。

#### 响应

该方法返回一个长时间运行的操作 `response`。我们将通过调用 `response.result()` 同步等待操作完成，该操作将阻塞直到模型部署完毕。如果这是第一次将模型部署到端点，则可能需要额外几分钟来完成资源的配置。

In [ ]:
DEPLOYED_NAME = "boston_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "disable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

## 发出在线预测请求

现在对您部署的模型进行在线预测。

获取测试项目

您将使用数据集的测试（留用）部分中的示例作为测试项目。

In [ ]:
test_item = x_test[0]
test_label = y_test[0]
print(test_item.shape)

### 发送预测请求

好的，现在你有了一个测试数据项。使用这个辅助函数 `predict_data`，它接受以下参数：

- `data`: 测试数据项，作为一个包含浮点数数值的 numpy 1D 数组。
- `endpoint`: Vertex 中的 `Endpoint` 资源的完全限定标识符，在这个资源中部署了 `Model` 资源。
- `parameters_dict`: 用于服务的额外参数。

这个函数使用预测客户端服务，并调用 `predict` 方法，使用以下参数：

- `endpoint`: Vertex 中的 `Endpoint` 资源的完全限定标识符，在这个资源中部署了 `Model` 资源。
- `instances`: 要预测的实例（数据项）列表。
- `parameters`: 用于服务的额外参数。

要将测试数据传递给预测服务，你需要将其打包上传到服务二进制文件中，步骤如下：

    1. 将数据项从 1D numpy 数组转换为 1D Python 列表。
    2. 将预测请求转换为序列化的 Google protobuf（`json_format.ParseDict()`）。

预测请求中的每个实例都是一个形如以下形式的字典条目：

                        {input_name: content}

- `input_name`: 底层模型输入层的名称。
- `content`: 数据项，作为 1D Python 列表。

由于 `predict()` 服务可以接受多个数据项（实例），你将发送单个数据项作为一个数据项列表。最后一步，你需要将数据项列表打包成 Google 的 protobuf 格式，这是我们传递给 `predict()` 服务的内容。

`response` 对象返回一个列表，列表中的每个元素对应请求中的相应图像。你会在每次预测的输出中看到：

- `predictions` -- 以 1K 美元为单位的预测房屋的中位数值。

In [ ]:
def predict_data(data, endpoint, parameters_dict):
    parameters = json_format.ParseDict(parameters_dict, Value())

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{serving_input: data.tolist()}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", prediction)


predict_data(test_item, endpoint_id, None)

## 下线`Model`资源

现在从提供服务的`Endpoint`资源中下线您的`Model`资源。使用这个辅助函数`undeploy_model`，它带有以下参数：

- `deployed_model_id`：当`Model`资源被部署时，端点服务返回的模型部署标识符。
- `endpoint`：`Model`被部署到的`Endpoint`资源的Vertex完全限定标识符。

此函数调用端点客户端服务的`undeploy_model`方法，带有以下参数：

- `deployed_model_id`：当`Model`资源被部署时，端点服务返回的模型部署标识符。
- `endpoint`：`Model`资源部署到的`Endpoint`资源的Vertex完全限定标识符。
- `traffic_split`：如何在`Endpoint`资源上的其余已部署模型之间分配流量。

由于这是`Endpoint`资源上唯一的已部署模型，您可以通过将`traffic_split`设置为空对象{}来简单地保持空。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理工作

要清理此项目中使用的所有GCP资源，您可以删除您用于教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME